In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import tensorflow as tf
import pickle

from datetime import datetime

from complexity_regularized_dcca.data.mnist import MNISTData
from complexity_regularized_dcca.algorithms.correlation import CCA
from complexity_regularized_dcca.algorithms.clustering import spectral_clustering_acc

from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC as SVM
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from complexity_regularized_dcca.algorithms.losses_metrics import MetricDict, MovingMetric
from tqdm.auto import tqdm

# Dataprovider

In [9]:
dataprovider = MNISTData.from_saved_data()

# Raw

In [ ]:
data_for_acc = dataprovider.test_data

outputs_met, labels_met = MetricDict(), MetricDict()
for data in data_for_acc:
    outputs_met.update(dict(view_0=data['nn_input_0']))
    labels_met.update(dict(labels=data["labels"].numpy()))

netw_output = outputs_met.output()
labels = labels_met.output()["labels"]

latent_repr = netw_output["view_0"]

acc = spectral_clustering_acc(
    data_points=latent_repr,
    labels=labels,
    num_classes=dataprovider.num_classes
)

print(acc)
print("---")

# PCA

In [4]:
pca_train_met = MetricDict()
for batch in dataprovider.training_data:
    pca_train_met.update(batch)
    
pca_train = pca_train_met.output()
    
data_for_acc = dataprovider.eval_data

for pca_dim in tqdm([80, 75, 70, 65, 60, 55, 50, 45, 40, 35, 30, 25, 20, 15, 10, 5]):
    
    pca = PCA(n_components=pca_dim, random_state=333).fit(pca_train['nn_input_0'])

    outputs_met, labels_met = MetricDict(), MetricDict()
    for data in data_for_acc:
        pca_transformed = pca.transform(data['nn_input_0'])

        outputs_met.update(dict(pca_view_0=pca_transformed))
        labels_met.update(dict(labels=data["labels"].numpy()))

    netw_output = outputs_met.output()
    labels = labels_met.output()["labels"]

    latent_repr = netw_output["pca_view_0"]

    acc = spectral_clustering_acc(
        data_points=latent_repr,
        labels=labels,
        num_classes=dataprovider.num_classes
    )

    print(pca_dim)
    print(acc)
    print("---")

  0%|          | 0/16 [00:00<?, ?it/s]

80
0.3261
---
75
0.3266
---
70
0.3281
---
65
0.3286
---
60
0.3294
---
55
0.3292
---
50
0.3306
---
45
0.3303
---
40
0.3267
---
35
0.3244
---
30
0.3216
---
25
0.3169
---
20
0.3126
---
15
0.32
---
10
0.2907
---
5
0.294
---


In [5]:
data_for_acc = dataprovider.test_data

pca_dim = 50
    
pca = PCA(n_components=pca_dim, random_state=333).fit(pca_train['nn_input_0'])

outputs_met, labels_met = MetricDict(), MetricDict()
for data in data_for_acc:
    pca_transformed = pca.transform(data['nn_input_0'])

    outputs_met.update(dict(pca_view_0=pca_transformed))
    labels_met.update(dict(labels=data["labels"].numpy()))

netw_output = outputs_met.output()
labels = labels_met.output()["labels"]

latent_repr = netw_output["pca_view_0"]

acc = spectral_clustering_acc(
    data_points=latent_repr,
    labels=labels,
    num_classes=dataprovider.num_classes
)

print(acc)

0.3807


# CCA

In [ ]:
dataprovider = MNISTData.from_saved_data('mnist_0boxes')

In [4]:
logfile = ''

cca_train_met = MetricDict()
for batch in dataprovider.training_data:
    cca_train_met.update(batch)
    
cca_train = cca_train_met.output()
    
data_for_acc = dataprovider.eval_data

results_dict = dict()
for cca_dim in tqdm([80, 75, 70, 65, 60, 55, 50, 45, 40, 35, 30, 25, 20, 15, 10, 5]):
    results_dict[cca_dim] = dict()
    for reg in tqdm([1e-4, 1e-3, 1e-2, 1e-1, 1]):
        B1, B2, epsilon, omega, ccor, mean_v0, mean_v1 = CCA(
            cca_train['nn_input_0'],
            cca_train['nn_input_1'],
            num_shared_dim=cca_dim,
            r1=reg,
            r2=reg
        )    

        outputs_met, labels_met = MetricDict(), MetricDict()
        for data in data_for_acc:
            m = tf.cast(tf.shape(data['nn_input_0'])[:1], tf.float32)
            v0_bar = tf.subtract(tf.cast(data['nn_input_0'], tf.float32), mean_v0) 
            v1_bar = tf.subtract(tf.cast(data['nn_input_1'], tf.float32), mean_v1)
            epsilon = B1@tf.transpose(v0_bar)
            omega = B2@tf.transpose(v1_bar)

            cca_transformed_0 = epsilon.numpy().T
            cca_transformed_1 = omega.numpy().T

            outputs_met.update(dict(cca_view_0=cca_transformed_0, cca_view_1=cca_transformed_1))
            labels_met.update(dict(labels=data["labels"].numpy()))

        netw_output = outputs_met.output()
        labels = labels_met.output()["labels"]

        latent_repr = netw_output["cca_view_0"]

        acc = spectral_clustering_acc(
            data_points=latent_repr,
            labels=labels,
            num_classes=dataprovider.num_classes
        )
        
        results_dict[cca_dim][reg] = acc
        
        with open(logfile, 'wb') as f:
            pickle.dump(results_dict, f)

        print(cca_dim)
        print(reg)
        print(acc)
        print("---")

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

2023-09-04 17:05:00.287440: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-09-04 17:05:00.301959: I tensorflow/core/util/cuda_solvers.cc:180] Creating CudaSolver handles for stream 0x5644368926a0


80
0.0001
0.4374
---
80
0.001
0.4267
---
80
0.01
0.4634
---
80
0.1
0.6226
---
80
1
0.7033
---


  0%|          | 0/5 [00:00<?, ?it/s]

...

15
0.0001
0.7492
---
15
0.001
0.767
---
15
0.01
0.7863
---
15
0.1
0.8048
---
15
1
0.7522
---


  0%|          | 0/5 [00:00<?, ?it/s]

10
0.0001
0.8264
---
10
0.001
0.7909
---
10
0.01
0.8103
---
10
0.1
0.8528
---
10
1
0.8386
---


  0%|          | 0/5 [00:00<?, ?it/s]

5
0.0001
0.6825
---
5
0.001
0.6867
---
5
0.01
0.6909
---
5
0.1
0.6763
---
5
1
0.6495
---


In [5]:
dataprovider = MNISTData.from_saved_data('mnist_1boxes')

In [6]:
logfile = ''

cca_train_met = MetricDict()
for batch in dataprovider.training_data:
    cca_train_met.update(batch)
    
cca_train = cca_train_met.output()
    
data_for_acc = dataprovider.eval_data

results_dict = dict()
for cca_dim in tqdm([80, 75, 70, 65, 60, 55, 50, 45, 40, 35, 30, 25, 20, 15, 10, 5]):
    results_dict[cca_dim] = dict()
    for reg in tqdm([1e-4, 1e-3, 1e-2, 1e-1, 1]):
        B1, B2, epsilon, omega, ccor, mean_v0, mean_v1 = CCA(
            cca_train['nn_input_0'],
            cca_train['nn_input_1'],
            num_shared_dim=cca_dim,
            r1=reg,
            r2=reg
        )    

        outputs_met, labels_met = MetricDict(), MetricDict()
        for data in data_for_acc:
            m = tf.cast(tf.shape(data['nn_input_0'])[:1], tf.float32)
            v0_bar = tf.subtract(tf.cast(data['nn_input_0'], tf.float32), mean_v0) 
            v1_bar = tf.subtract(tf.cast(data['nn_input_1'], tf.float32), mean_v1)
            epsilon = B1@tf.transpose(v0_bar)
            omega = B2@tf.transpose(v1_bar)

            cca_transformed_0 = epsilon.numpy().T
            cca_transformed_1 = omega.numpy().T

            outputs_met.update(dict(cca_view_0=cca_transformed_0, cca_view_1=cca_transformed_1))
            labels_met.update(dict(labels=data["labels"].numpy()))

        netw_output = outputs_met.output()
        labels = labels_met.output()["labels"]

        latent_repr = netw_output["cca_view_0"]

        acc = spectral_clustering_acc(
            data_points=latent_repr,
            labels=labels,
            num_classes=dataprovider.num_classes
        )
        
        results_dict[cca_dim][reg] = acc
        
        with open(logfile, 'wb') as f:
            pickle.dump(results_dict, f)

        print(cca_dim)
        print(reg)
        print(acc)
        print("---")

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

80
0.0001
0.3911
---
80
0.001
0.4403
---
80
0.01
0.4277
---
80
0.1
0.3878
---
80
1
0.4253
---


  0%|          | 0/5 [00:00<?, ?it/s]

75
0.0001
0.4334
---
75
0.001
0.4425
---
75
0.01
0.4302
---
75
0.1
0.3928
---
75
1
0.4284
---


...

15
0.0001
0.6524
---
15
0.001
0.6645
---
15
0.01
0.6809
---
15
0.1
0.6982
---
15
1
0.6791
---


  0%|          | 0/5 [00:00<?, ?it/s]

10
0.0001
0.6721
---
10
0.001
0.6821
---
10
0.01
0.6911
---
10
0.1
0.7086
---
10
1
0.717
---


  0%|          | 0/5 [00:00<?, ?it/s]

5
0.0001
0.5474
---
5
0.001
0.5537
---
5
0.01
0.5793
---
5
0.1
0.5746
---
5
1
0.5535
---


In [7]:
dataprovider = MNISTData.from_saved_data('mnist_2boxes')

In [8]:
logfile = ''

cca_train_met = MetricDict()
for batch in dataprovider.training_data:
    cca_train_met.update(batch)
    
cca_train = cca_train_met.output()
    
data_for_acc = dataprovider.eval_data

results_dict = dict()
for cca_dim in tqdm([80, 75, 70, 65, 60, 55, 50, 45, 40, 35, 30, 25, 20, 15, 10, 5]):
    results_dict[cca_dim] = dict()
    for reg in tqdm([1e-4, 1e-3, 1e-2, 1e-1, 1]):
        B1, B2, epsilon, omega, ccor, mean_v0, mean_v1 = CCA(
            cca_train['nn_input_0'],
            cca_train['nn_input_1'],
            num_shared_dim=cca_dim,
            r1=reg,
            r2=reg
        )    

        outputs_met, labels_met = MetricDict(), MetricDict()
        for data in data_for_acc:
            m = tf.cast(tf.shape(data['nn_input_0'])[:1], tf.float32)
            v0_bar = tf.subtract(tf.cast(data['nn_input_0'], tf.float32), mean_v0) 
            v1_bar = tf.subtract(tf.cast(data['nn_input_1'], tf.float32), mean_v1)
            epsilon = B1@tf.transpose(v0_bar)
            omega = B2@tf.transpose(v1_bar)

            cca_transformed_0 = epsilon.numpy().T
            cca_transformed_1 = omega.numpy().T

            outputs_met.update(dict(cca_view_0=cca_transformed_0, cca_view_1=cca_transformed_1))
            labels_met.update(dict(labels=data["labels"].numpy()))

        netw_output = outputs_met.output()
        labels = labels_met.output()["labels"]

        latent_repr = netw_output["cca_view_0"]

        acc = spectral_clustering_acc(
            data_points=latent_repr,
            labels=labels,
            num_classes=dataprovider.num_classes
        )
        
        results_dict[cca_dim][reg] = acc
        
        with open(logfile, 'wb') as f:
            pickle.dump(results_dict, f)

        print(cca_dim)
        print(reg)
        print(acc)
        print("---")

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

80
0.0001
0.3688
---
80
0.001
0.4327
---
80
0.01
0.4584
---
80
0.1
0.3832
---
80
1
0.3476
---


  0%|          | 0/5 [00:00<?, ?it/s]

75
0.0001
0.3874
---
75
0.001
0.4316
---
75
0.01
0.5077
---
75
0.1
0.4269
---
75
1
0.3505
---


...

15
0.0001
0.5517
---
15
0.001
0.5908
---
15
0.01
0.6364
---
15
0.1
0.6375
---
15
1
0.5481
---


  0%|          | 0/5 [00:00<?, ?it/s]

10
0.0001
0.5838
---
10
0.001
0.6067
---
10
0.01
0.6552
---
10
0.1
0.6603
---
10
1
0.6457
---


  0%|          | 0/5 [00:00<?, ?it/s]

5
0.0001
0.4986
---
5
0.001
0.4995
---
5
0.01
0.495
---
5
0.1
0.5118
---
5
1
0.4934
---


# Test data

cca_train_met = MetricDict()
for batch in dataprovider.training_data:
    cca_train_met.update(batch)
    
cca_train = cca_train_met.output()

data_for_acc = dataprovider.test_data

cca_dim = 
reg = 

B1, B2, epsilon, omega, ccor, mean_v0, mean_v1 = CCA(
    cca_train['nn_input_0'],
    cca_train['nn_input_1'],
    num_shared_dim=cca_dim,
    r1=reg,
    r2=reg
)    

outputs_met, labels_met = MetricDict(), MetricDict()
for data in data_for_acc:
    m = tf.cast(tf.shape(data['nn_input_0'])[:1], tf.float32)
    v0_bar = tf.subtract(tf.cast(data['nn_input_0'], tf.float32), mean_v0) 
    v1_bar = tf.subtract(tf.cast(data['nn_input_1'], tf.float32), mean_v1)
    epsilon = B1@tf.transpose(v0_bar)
    omega = B2@tf.transpose(v1_bar)

    cca_transformed_0 = epsilon.numpy().T
    cca_transformed_1 = omega.numpy().T

    outputs_met.update(dict(cca_view_0=cca_transformed_0, cca_view_1=cca_transformed_1))
    labels_met.update(dict(labels=data["labels"].numpy()))

netw_output = outputs_met.output()
labels = labels_met.output()["labels"]

latent_repr = netw_output["cca_view_0"]

acc = spectral_clustering_acc(
    data_points=latent_repr,
    labels=labels,
    num_classes=dataprovider.num_classes
)

print(cca_dim)
print(reg)
print(acc)
print("---")